In [8]:
pip install tf-keras

In [1]:
# 📌 Step 2: Import Libraries
import psycopg2
import pandas as pd
import os

# 📌 Step 3: Set Up Database Connection (Replace with Your Details)
DATABASE_URL = "dbname='postgres' user='postgres' host='34.59.119.208' password='avantichhaya'"

def connect_db():
    try:
        conn = psycopg2.connect(DATABASE_URL)
        print("✅ Connected to PostgreSQL Cloud SQL")
        return conn
    except Exception as e:
        print("🚨 Connection Error:", e)
        return None

conn = connect_db()
cursor = conn.cursor()


✅ Connected to PostgreSQL Cloud SQL


cursor.execute("""
    UPDATE emails
    SET classified_category = NULL,
        escalated = NULL,
        classification_confidence = NULL;
""")
conn.commit()

In [43]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where classified_category = None;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_52216\2600313899.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where classified_category = None;", conn)


DatabaseError: Execution failed on sql 'SELECT * FROM emails where classified_category = None;': column "none" does not exist
LINE 1: SELECT * FROM emails where classified_category = None;
                                                         ^


In [49]:
from transformers import pipeline

# Step 1: Setup classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Step 2: Define labels
label_map = {
    "Leave Request": "Requests related to taking leave or vacation",
    "Onboarding": "Questions about new hire onboarding or joining formalities",
    "Job Offer": "Inquiries regarding job offers or employment contracts",
    "Payroll Inquiry": "Questions related to salary, payslips, or payroll processing",
    "Benefits Inquiry": "Questions about insurance, medical, or employee benefits",
    "Resignation & Exit": "Emails about resignation, exit process, or final settlements",
    "Attendance & Timesheet": "Issues about work hours, attendance or timesheets",
    "Recruitment Process": "Questions about interview, screening or hiring stages",
    "Policy Clarification": "Clarification about company policies or procedures",
    "Training & Development": "Queries about training programs or skill development",
    "Work From Home Requests": "Requests or updates regarding remote work",
    "Relocation & Transfer": "Inquiries about internal transfers or relocation",
    "Expense Reimbursement": "Questions about reimbursements or expense claims",
    "IT & Access Issues": "Issues about system access, accounts, or technical problems",
    "Events & Celebrations": "Emails about office events, parties, or celebrations"
}

descriptive_labels = list(label_map.values())

#############################################
# Step 3: Fetch unclassified emails
#############################################

cursor.execute("""
    SELECT email_id, subject, body, thread_id, received_at
    FROM emails
    WHERE classified_category IS NULL
    ORDER BY received_at;
""")
emails_to_classify = cursor.fetchall()

print(f"🔎 Found {len(emails_to_classify)} unclassified emails.")

#############################################
# Step 4: Classify and update category + confidence
#############################################

for email_id, subject, body, thread_id, received_at in emails_to_classify:

    # --- Build context ---
    cursor.execute("""
        SELECT subject, body FROM emails
        WHERE thread_id = %s AND received_at < %s
        ORDER BY received_at;
    """, (thread_id, received_at))
    previous_emails = cursor.fetchall()

    context = ""
    for prev_subj, prev_body in previous_emails:
        context += f"Subject: {prev_subj}\nBody: {prev_body}\n---\n"

    email_text = context + f"Subject: {subject}\nBody: {body}"

    # --- Classification ---
    result = classifier(email_text, descriptive_labels)
    predicted_description = result["labels"][0]
    confidence = result["scores"][0]

    ############################
    # ✅ Safe label matching — no escalation here!
    ############################
    predicted_label = None

    for key, value in label_map.items():
        if value.strip() == predicted_description.strip():
            predicted_label = key
            break

    if predicted_label is None:
        print(f"⚠️ No label found matching '{predicted_description}'. Assigning UNKNOWN category.")
        predicted_label = "Unknown"

    print(f"📧 Email ID {email_id} → Category: {predicted_label}, Confidence: {round(confidence, 2)}")

    # --- Update DB ---
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s,
            classification_confidence = %s
        WHERE email_id = %s;
    """, (predicted_label, confidence, email_id))

    # --- IMMEDIATE CONFIRMATION ---
    cursor.execute("""
        SELECT classified_category, classification_confidence
        FROM emails
        WHERE email_id = %s;
    """, (email_id,))
    updated = cursor.fetchone()
    print(f"📝 Confirmed update → Email ID {email_id}: Category = {updated[0]}, Confidence = {updated[1]}")

# --- Commit ALL updates ---
conn.commit()

print("✅ Classification and confidence updated for all emails.")


Device set to use cpu


🔎 Found 0 unclassified emails.
✅ Classification and confidence updated for all emails.


In [47]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where classification_confidence is null;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_52216\4257764326.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where classification_confidence is null;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence,response_email,learn,similarity_source_email


In [55]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where classification_confidence<0.2;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_52216\2026230170.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where classification_confidence<0.2;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence,response_email,learn,similarity_source_email,similarity_to_learned
0,25,newsletters@audible.com,90% off plus a bonus $20 credit.,audible https://www.audible.com/sign-in?rdpath...,2024-11-27 09:44:15.000000,Policy Clarification,NOT RESPONDED,None,NaN,0.129796,None,None,140,0.168419
1,19,google-gemini-noreply@google.com,"Hey Chhau, welcome to Gemini Advanced",Get the latest updates and more. Subscribe to ...,2025-05-03 20:02:02.000000,Job Offer,NEW,None,NaN,0.087945,None,None,137,0.352825
2,21,avanaya3@gmail.com,Urgent,I need to do this blah blah.,2025-05-04 03:24:37.000000,Expense Reimbursement,RESPONDED,None,NaN,0.133779,None,None,137,0.404424
3,41,chandratreavanti@gmail.com,help needed,"Hello, how are you? I was wondering how to res...",2025-05-05 00:14:25.000000,IT & Access Issues,NOT RESPONDED,None,NaN,0.139094,None,None,137,0.497989
4,139,user39@example.com,Interview reschedule request,I need to reschedule my interview.,2025-03-24 17:52:00.000000,Recruitment Process,pending,None,218.0,0.176617,None,None,140,0.582784
5,34,email@market.temuemail.com,A massive thank you! $6 OFF for you!,----------------------------------------------...,2024-11-20 01:17:09.000000,Leave Request,NOT RESPONDED,None,NaN,0.199713,None,None,137,0.345556
6,30,email@market.temuemail.com,"Your Purchase, Our Thanks!",----------------------------------------------...,2025-05-04 00:14:43.392800,Leave Request,NOT RESPONDED,None,NaN,0.198693,None,None,137,0.389688
7,112,user12@example.com,Negotiating offer terms,I would like to discuss the offered compensati...,2025-03-31 11:32:00.000000,Job Offer,pending,None,206.0,0.179425,None,None,140,0.377501
8,27,email@market.temuemail.com,Your loyalty has paid off!,----------------------------------------------...,2024-11-27 01:10:05.000000,Leave Request,NOT RESPONDED,None,NaN,0.190893,None,False,137,0.371920
9,140,user40@example.com,Job application withdrawal,I would like to withdraw my application.,2025-03-23 10:56:00.000000,Resignation & Exit,pending,None,219.0,0.143597,None,True,140,1.000000


Fetch low-confidence emails + compute similarity to learned vectors (only compute)

In [54]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

ZS_threshold = 0.2
similarity_threshold = 0.75

encoder = SentenceTransformer('all-mpnet-base-v2')

#############################################
# 1️⃣ Load learned vectors
#############################################

cursor.execute("""
    SELECT email_id, category, vector FROM learned_email_vectors;
""")
learned_rows = cursor.fetchall()

learned_vectors = []
learned_labels = []
learned_email_ids = []

for email_id, category, vector in learned_rows:
    learned_vectors.append(vector)
    learned_labels.append(category)
    learned_email_ids.append(email_id)

if learned_vectors:
    learned_vectors = np.vstack(learned_vectors)
    print(f"✅ Loaded {len(learned_vectors)} learned vectors.")
else:
    learned_vectors = None
    print("⚠️ No learned vectors available.")

#############################################
# 2️⃣ Fetch low-confidence emails
#############################################

cursor.execute("""
    SELECT email_id, subject, body
    FROM emails
    WHERE classification_confidence < %s
      AND classified_category IS NOT NULL
      AND classified_category != 'human_intervention';
""", (ZS_threshold, ))

low_confidence_emails = cursor.fetchall()

print(f"🔎 Found {len(low_confidence_emails)} low-confidence emails.")

#############################################
# 3️⃣ Compute similarity & update DB (ONLY similarity fields)
#############################################

for email_id, subject, body in low_confidence_emails:

    email_text = f"Subject: {subject}\nBody: {body}"

    similarity_score = None
    similarity_source_email = None

    if learned_vectors is not None:

        email_vector = encoder.encode(email_text)
        sims = cosine_similarity([email_vector], learned_vectors)[0]

        max_sim = sims.max()
        max_sim_index = sims.argmax()

        similarity_score = max_sim
        similarity_source_email = learned_email_ids[max_sim_index]

        print(f"Email ID {email_id} → Similarity {round(max_sim, 2)} to email_id {similarity_source_email} (Category: {learned_labels[max_sim_index]})")

    else:
        print(f"Email ID {email_id} → No learned vectors available. Similarity = None.")

    # ✅ Update only similarity fields in DB
    cursor.execute("""
        UPDATE emails
        SET similarity_to_learned = %s,
            similarity_source_email = %s
        WHERE email_id = %s;
    """, (similarity_score, similarity_source_email, email_id))

    conn.commit()



✅ Loaded 2 learned vectors.
🔎 Found 12 low-confidence emails.
Email ID 25 → Similarity 0.17 to email_id 140 (Category: Onboarding)
Email ID 19 → Similarity 0.35 to email_id 137 (Category: Recruitment Process)
Email ID 21 → Similarity 0.4 to email_id 137 (Category: Recruitment Process)
Email ID 41 → Similarity 0.5 to email_id 137 (Category: Recruitment Process)
Email ID 139 → Similarity 0.58 to email_id 140 (Category: Onboarding)
Email ID 34 → Similarity 0.35 to email_id 137 (Category: Recruitment Process)
Email ID 30 → Similarity 0.39 to email_id 137 (Category: Recruitment Process)
Email ID 112 → Similarity 0.38 to email_id 140 (Category: Onboarding)
Email ID 27 → Similarity 0.37 to email_id 137 (Category: Recruitment Process)
Email ID 140 → Similarity 1.0 to email_id 140 (Category: Onboarding)
Email ID 137 → Similarity 1.0 to email_id 137 (Category: Recruitment Process)
Email ID 177 → Similarity 0.38 to email_id 137 (Category: Recruitment Process)


In [56]:
similarity_threshold = 0.75

# Step 3: Fetch emails where similarity has been computed (but escalation not yet updated)
cursor.execute("""
    SELECT email_id, similarity_to_learned, similarity_source_email
    FROM emails
    WHERE classification_confidence < %s
      AND classified_category IS NOT NULL
      AND classified_category != 'human_intervention'
      AND similarity_to_learned IS NOT NULL;
""", (ZS_threshold, ))

emails_for_decision = cursor.fetchall()

print(f"🔎 Found {len(emails_for_decision)} emails to review for escalation or category update.")

#############################################
# For each email → decide escalation or assign learned category
#############################################

for email_id, similarity_score, similarity_source_email in emails_for_decision:

    # Default escalation to False
    escalate = False

    if similarity_score >= similarity_threshold:
        # --- Similar enough → assign learned category ---
        cursor.execute("""
            SELECT category FROM learned_email_vectors
            WHERE email_id = %s;
        """, (similarity_source_email,))
        similar_category_row = cursor.fetchone()

        if similar_category_row:
            final_category = similar_category_row[0]
            escalate = False
            print(f"✅ Email ID {email_id}: Assigned learned category → {final_category} (Similarity: {round(similarity_score, 2)})")
        else:
            # Fallback — no category found
            final_category = "Unknown"
            escalate = True
            print(f"⚠️ Email ID {email_id}: Similar email found but no category assigned. Escalating.")
    else:
        # --- Similarity too low → escalate ---
        final_category = "human_intervention"
        escalate = True
        print(f"⚠️ Email ID {email_id}: Similarity too low ({round(similarity_score, 2)}). Escalated to human_intervention.")

    # --- Update the email ---
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s,
            escalated = %s
        WHERE email_id = %s;
    """, (final_category, escalate, email_id))

    # Confirm the update
    cursor.execute("""
        SELECT classified_category, escalated
        FROM emails
        WHERE email_id = %s;
    """, (email_id,))
    check = cursor.fetchone()
    print(f"📝 Updated Email ID {email_id}: Final Category = {check[0]}, Escalated = {check[1]}")

conn.commit()
print("✅ All emails updated with final category and escalation status.")


🔎 Found 12 emails to review for escalation or category update.
⚠️ Email ID 25: Similarity too low (0.17). Escalated to human_intervention.
📝 Updated Email ID 25: Final Category = human_intervention, Escalated = True
⚠️ Email ID 19: Similarity too low (0.35). Escalated to human_intervention.
📝 Updated Email ID 19: Final Category = human_intervention, Escalated = True
⚠️ Email ID 21: Similarity too low (0.4). Escalated to human_intervention.
📝 Updated Email ID 21: Final Category = human_intervention, Escalated = True
⚠️ Email ID 41: Similarity too low (0.5). Escalated to human_intervention.
📝 Updated Email ID 41: Final Category = human_intervention, Escalated = True
⚠️ Email ID 139: Similarity too low (0.58). Escalated to human_intervention.
📝 Updated Email ID 139: Final Category = human_intervention, Escalated = True
⚠️ Email ID 34: Similarity too low (0.35). Escalated to human_intervention.
📝 Updated Email ID 34: Final Category = human_intervention, Escalated = True
⚠️ Email ID 30: Sim

In [14]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails order by received_at desc limit 5;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_52216\3527725929.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails order by received_at desc limit 5;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence,response_email,learn
0,41,chandratreavanti@gmail.com,help needed,"Hello, how are you? I was wondering how to res...",2025-05-05 00:14:25,human_intervention,NOT RESPONDED,True,None,0.139094,None,None
1,38,no-reply@accounts.google.com,Security alert,[image: Google] App password created to sign i...,2025-05-04 19:37:35,None,NOT RESPONDED,False,None,0.329685,None,None
2,39,no-reply@accounts.google.com,2-Step Verification turned on,[image: Google] 2-Step Verification turned on ...,2025-05-04 19:37:06,None,NOT RESPONDED,False,None,0.259288,None,None
3,40,no-reply@accounts.google.com,Security alert,[image: Google] Phone number added for 2-Step ...,2025-05-04 19:37:05,None,NOT RESPONDED,False,None,0.214853,None,None
4,22,avanaya3@gmail.com,Fwd: Leave Request: Shake age field special none.,---------- Forwarded message --------- From: C...,2025-05-04 04:14:09,None,NOT RESPONDED,False,None,0.377153,None,None


In [57]:
# 📌 Step 6: Retrieve Data as DataFrame
df = pd.read_sql("SELECT * FROM emails where escalated=True;", conn)
df


C:\Users\chand\AppData\Local\Temp\ipykernel_52216\28204962.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM emails where escalated=True;", conn)


,email_id,sender_email,subject,body,received_at,classified_category,status,escalated,thread_id,classification_confidence,response_email,learn,similarity_source_email,similarity_to_learned
0,25,newsletters@audible.com,90% off plus a bonus $20 credit.,audible https://www.audible.com/sign-in?rdpath...,2024-11-27 09:44:15.000000,human_intervention,NOT RESPONDED,True,NaN,0.129796,None,None,140,0.168419
1,19,google-gemini-noreply@google.com,"Hey Chhau, welcome to Gemini Advanced",Get the latest updates and more. Subscribe to ...,2025-05-03 20:02:02.000000,human_intervention,NEW,True,NaN,0.087945,None,None,137,0.352825
2,21,avanaya3@gmail.com,Urgent,I need to do this blah blah.,2025-05-04 03:24:37.000000,human_intervention,RESPONDED,True,NaN,0.133779,None,None,137,0.404424
3,41,chandratreavanti@gmail.com,help needed,"Hello, how are you? I was wondering how to res...",2025-05-05 00:14:25.000000,human_intervention,NOT RESPONDED,True,NaN,0.139094,None,None,137,0.497989
4,139,user39@example.com,Interview reschedule request,I need to reschedule my interview.,2025-03-24 17:52:00.000000,human_intervention,pending,True,218.0,0.176617,None,None,140,0.582784
5,34,email@market.temuemail.com,A massive thank you! $6 OFF for you!,----------------------------------------------...,2024-11-20 01:17:09.000000,human_intervention,NOT RESPONDED,True,NaN,0.199713,None,None,137,0.345556
6,30,email@market.temuemail.com,"Your Purchase, Our Thanks!",----------------------------------------------...,2025-05-04 00:14:43.392800,human_intervention,NOT RESPONDED,True,NaN,0.198693,None,None,137,0.389688
7,112,user12@example.com,Negotiating offer terms,I would like to discuss the offered compensati...,2025-03-31 11:32:00.000000,Recruitment Process,pending,True,206.0,0.179425,None,True,140,0.377501
8,27,email@market.temuemail.com,Your loyalty has paid off!,----------------------------------------------...,2024-11-27 01:10:05.000000,human_intervention,NOT RESPONDED,True,NaN,0.190893,None,False,137,0.371920
9,177,user77@example.com,Assistance required,Im unsure who to contact regarding this issue.,2025-03-31 17:40:00.000000,human_intervention,pending,True,230.0,0.194488,None,None,137,0.380648
